In [3]:
#import libraries.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
import math

In [4]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [5]:
estimator_A = load_model('Models/Two_stage/trained_model19.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [9]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed4_sec.hdf5','r')
test_input_shape = h5f['mixed4_sec'].shape
h5f.close()
print(test_input_shape)

(257, 295522)


In [10]:
with h5py.File('TIMIT/Organized/Two_stage_sec_set/predicted_sec_set.hdf5', "w") as f:
    dset = f.create_dataset('predicted_sec_set',shape=(test_input_shape[0]*2,0), maxshape=(test_input_shape[0]*2,100000000000),
                            chunks=True)


In [12]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed4_sec.hdf5','r')
step = 128
for i in range(0, test_input_shape[1], step):
    if test_input_shape[1] - i < step:
        test_input0 = h5f['mixed4_sec'][:, i:]
    else:
        test_input0 = h5f['mixed4_sec'][:, i : i+128]
    test_input = estimator_A.predict(test_input0.T).T
    f = h5py.File('TIMIT/Organized/Two_stage_sec_set/predicted_sec_set.hdf5', 'r+')     # open the file
    dset = f['predicted_sec_set']                # load the data
    dset_shape = dset.shape[1]
    dset.resize(dset_shape + test_input.shape[1], axis=1)  
    dset[:,dset_shape:] = test_input
    f.close()  

h5f.close()

In [10]:
test_input.shape

(128, 514)